<a href="https://colab.research.google.com/github/ShirGrinblat/DataScience/blob/master/RNN_names_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN - many to one 
many = name letters, one = language 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install Unidecode

Load names from file, each file is a language 
constructing a dictionary in a form of:   {language: [names ...]}

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import unidecode

def findFiles(path): return glob.glob(path)
print(findFiles('/content/drive/My Drive/MusicProject/data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"

n_letters = len(all_letters)

def unicodeToAscii(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return lines
    #return [unicodeToAscii(line) for line in lines]

for filename in findFiles('/content/drive/My Drive/MusicProject/data/names/*.txt'):
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

['/content/drive/My Drive/MusicProject/data/names/Dutch.txt', '/content/drive/My Drive/MusicProject/data/names/Czech.txt', '/content/drive/My Drive/MusicProject/data/names/Greek.txt', '/content/drive/My Drive/MusicProject/data/names/Japanese.txt', '/content/drive/My Drive/MusicProject/data/names/Arabic.txt', '/content/drive/My Drive/MusicProject/data/names/Chinese.txt', '/content/drive/My Drive/MusicProject/data/names/Italian.txt', '/content/drive/My Drive/MusicProject/data/names/French.txt', '/content/drive/My Drive/MusicProject/data/names/German.txt', '/content/drive/My Drive/MusicProject/data/names/Irish.txt', '/content/drive/My Drive/MusicProject/data/names/Vietnamese.txt', '/content/drive/My Drive/MusicProject/data/names/Scottish.txt', '/content/drive/My Drive/MusicProject/data/names/Korean.txt', '/content/drive/My Drive/MusicProject/data/names/Russian.txt', '/content/drive/My Drive/MusicProject/data/names/Portuguese.txt', '/content/drive/My Drive/MusicProject/data/names/English.t

In [0]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [0]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    # print(letter)
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))
print(lineToTensor('Jones').size())
# print(letterToIndex('G'))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([5, 1, 57])


In [0]:
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.Softmax()

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return Variable(torch.zeros(1, self.hidden_size))

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

We want to predict what is the origin of Albert

In [0]:
input = Variable(letterToTensor('A'))
hidden = Variable(torch.zeros(1, n_hidden))

output, next_hidden = rnn(input, hidden)

input = Variable(letterToTensor('l'))
output, next_hidden = rnn(input, next_hidden)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
input = Variable(lineToTensor('Albert'))
hidden = Variable(torch.zeros(1, n_hidden))

output, next_hidden = rnn(input[0], hidden)
print(output)

tensor([[0.0544, 0.0622, 0.0566, 0.0498, 0.0548, 0.0512, 0.0523, 0.0553, 0.0562,
         0.0526, 0.0584, 0.0605, 0.0573, 0.0519, 0.0565, 0.0571, 0.0563, 0.0566]],
       grad_fn=<SoftmaxBackward>)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


# training

In [0]:
def categoryFromOutput(output):
    top_n, top_i = output.data.topk(1) # Tensor out of Variable with .data
    category_i = top_i[0][0]
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('Czech', tensor(1))


In [0]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.LongTensor([all_categories.index(category)])
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = Polish / line = Kumiega
category = English / line = Wolfe
category = Japanese / line = Takekawa
category = Greek / line = Close
category = Greek / line = Matsoukis
category = Irish / line = Macdermott
category = Russian / line = Avalov
category = Spanish / line = Fuentes
category = Japanese / line = Sakiyurai
category = English / line = Mackenzie


In [0]:
criterion = nn.NLLLoss()
# If you set this too high, it might explode. If too low, it might not learn
learning_rate = 0.005

# category_tensor: language, line_tensor: surname
def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

In [0]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100,
                                                timeSince(start), loss, line,
                                                guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


5000 5% (0m 7s) -0.0558 Koumans / Greek ✗ (Dutch)
10000 10% (0m 14s) -0.0602 Lefévre / Scottish ✗ (French)
15000 15% (0m 21s) -0.0614 Kelly / Portuguese ✗ (Scottish)
20000 20% (0m 29s) -0.0572 Reynders / Greek ✗ (Dutch)
25000 25% (0m 36s) -0.0612 Mertens / Greek ✗ (Dutch)
30000 30% (0m 44s) -0.0616 Ślązak / Greek ✗ (Polish)
35000 35% (0m 51s) -0.0625 Ramires / Greek ✗ (Portuguese)
40000 40% (0m 58s) -0.0448 Chang / Greek ✗ (Korean)
45000 45% (1m 5s) -0.0684 Hao / Italian ✗ (Chinese)
50000 50% (1m 13s) -0.0448 Schmidt / Greek ✗ (German)
55000 55% (1m 20s) -0.0378 Moulin / Italian ✗ (French)
60000 60% (1m 27s) -0.0613 Ferro / Italian ✗ (Portuguese)
65000 65% (1m 35s) -0.0492 Aswad / Chinese ✗ (Arabic)
70000 70% (1m 42s) -0.6449 Zhu / Chinese ✓
75000 75% (1m 49s) -0.0093 Iñíguez / Polish ✗ (Spanish)
80000 80% (1m 57s) -0.0013 Róg / Chinese ✗ (Polish)
85000 85% (2m 4s) -0.0031 Tong / Chinese ✗ (Vietnamese)
90000 90% (2m 11s) -0.0039 Delgado / Italian ✗ (Portuguese)
95000 95% (2m 19s) -0.00

In [0]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.show(all_losses)

<Figure size 432x288 with 0 Axes>

In [0]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])
# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output


In [0]:
predict('Miguel')


> Miguel
(0.39) German
(0.32) Arabic
(0.09) Polish


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
